In [1]:
import pandas as pd
import uuid

In [2]:
!pip install openpyxl


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#Crear dimensiones base para variable a nivel sujeto
temaInteres = pd.DataFrame(columns=['id','nombre','descripcion'])

etapa= pd.DataFrame(columns=['id','numEtapa','nombreAnalisis','nombreBD','descripcion',
                             'eventoInicial','eventoFinal'])

fecha=pd.DataFrame(columns=['dia','mes','anio'])

evento=pd.DataFrame(columns=['id','nombre','descripcion'])

variable=pd.DataFrame(columns=['id','nombreBD','nombreAnalisis','unidades','edadCorregida','derivada',
                               'impacto','tipoDato','tipoVariable','nivelMedicion','longitudinal',
                               'descripcionCorta','descripcionLarga'])


In [4]:
# Conseguir dataframes de diccionario
file_dict=pd.ExcelFile('data/KMC-diccionario.xlsx')
df_etapas=pd.read_excel(file_dict, sheet_name='ETAPAS')
df_temas=pd.read_excel(file_dict, sheet_name='Temas de Interés')
df_variables=pd.read_excel(file_dict, sheet_name='KMC-12mEC-DIC-VAR-version2024')

In [5]:
#Poblar dimensión de eventos
eventos = pd.unique(
    pd.concat([
        df_etapas['Evento inicial (id-var)'],
        df_etapas['Evento-final (id-var)']
    ])
).tolist()

ids_eventos = [str(uuid.uuid4()) for _ in range(len(eventos))]

for i in range(len(eventos)):
    evento.loc[len(evento)] = [
        ids_eventos[i],
        eventos[i],
        ""
    ]

In [6]:
#Poblar dimensión de etapas
def encontrar_evento(nombre):
    fila=evento[evento['nombre']==nombre]
    return fila['id'].values[0]

ids_etapas = [str(uuid.uuid4()) for _ in range(len(df_etapas))]

for index, row in df_etapas.iterrows():
    etapa.loc[row['NUMERO ETAPA']]=[
        ids_etapas[index],
        row['NUMERO ETAPA'],
        row['ID-ETAPA'].lower(),
        row['NOMBRE CORTO'].lower(),
        row['DESCRIPCION'].lower(),
        encontrar_evento(row['Evento inicial (id-var)']),
        encontrar_evento(row['Evento-final (id-var)'])
    ]

In [7]:
#Poblar dimensión tema de interes
ids_temas = [str(uuid.uuid4()) for _ in range(len(df_temas))]

for index,row in df_temas.iterrows():
    temaInteres.loc[row['Número TdI']]=[
        ids_temas[index],
        row['ID-TdI'],
        row['DescripciOn']
    ]

In [8]:
#Poblar dimension fecha
fecha.loc[0]=[
    1,
    1,
    2025
]

In [9]:
#Poblar dimension variable

ids_vars = [str(uuid.uuid4()) for _ in range(len(df_variables))]

for index,row in df_variables.iterrows():
    variable.loc[index]=[
        ids_vars[index],
        row['NOMBRE EN LA BdeD'],
        row['VAR-ID'],
        row['UNITS'],
        False, #TODO: Edad coregida:De donde se saca esto?
        False, #TODO: Derivada: De donde se saca esto?
        False, #TODO: Impacto: ¿?
        row['VAR-TYPE'],
        False, #Tipo Variable
        False, #Nivel medicion
        False, #Longitudinal
        row['VAR-SHORT DESCRIPTION'],
        row['VAR-LONG DESCRIPTION']
    ]

In [22]:
#Crear diccionario de valor categorias
cols = [
    "VAL1", "SHORT-NAME1", "VAL2", "SHORT-NAME2",
    "VAL3", "SHORT-NAME3", "VAL4", "SHORT-NAME4", "VAL5", "SHORT-NAME5",
    "VAL6", "SHORT-NAME6", "VAL7", "SHORT-NAME7", "VAL8", "SHORT-NAME8",
    "VAL9", "SHORT-NAME9", "VAL10", "SHORT-NAME10", "VAL11", "SHORT-NAME11"
]

df_categorias=df_variables[cols]

unique_dicts = set()

def add_unique_dict(d):
    frozen = frozenset(d.items())
    if frozen not in unique_dicts:
        unique_dicts.add(frozen)

for index, row in df_categorias.iterrows():
    for i, col in  enumerate(df_categorias.columns):
        
        if i + 1 < len(df_categorias.columns):  
            siguiente_col = df_categorias.columns[i + 1]
        if pd.notna(row[col]) and 'VAL' in col and row[col] != ' ':
            dicc={
                'value':int(row[col]),
                'description':str(row[siguiente_col]).strip()
            }
            add_unique_dict(dicc)


In [ ]:
valorCategoria=pd.DataFrame(columns=['id','codificacion','descripcion'])
for d in unique_dicts:
    data = dict(d) 
    nueva_fila = {
        'id': str(uuid.uuid4()),               
        'codificacion': data['value'],         
        'descripcion': data['description']     
    }
    valorCategoria.loc[len(valorCategoria)] = nueva_fila

valorCategoria

,id,codificacion,descripcion
0,c863b2b7-bc24-4566-93eb-a470fc999205,2,Contributivo
1,09755c93-3409-418e-839f-437d99b8d3fc,14,Astigmatismo Hipermetrópico
2,3c7feb66-c38c-4e56-874e-1435f3123df9,1,ADOLESCENTE
3,f0a88778-453c-4d3c-884e-823c48ab9d12,3,Casa
4,5a70d915-c22e-41a2-bcc5-9ac6b674dc78,3,AltoRiesgo (más de 2100 gr y 37 semEG o más)
...,...,...,...
311,cbf23e42-b687-4b12-9897-ac33fb5445a3,0,"NADA (por ahora, o no reportado)"
312,d5a55cfd-97f5-49c0-90cf-e8a328b5e3d7,1,Displasia Broncopulmonar
313,7580bd2e-ced2-44d3-8ccd-5f1843ab592f,3,Recuperándose
314,f626e04a-bbae-46e4-8131-63e9ba5efb8e,1,ESPONTÁNEA
